Michelle 2/20/19

Replacement of Pigeon.jl w/o ROS requirements

In [1]:
using LinearAlgebra
using Distributed
using StaticArrays
using DifferentialDynamicsModels
using LinearDynamicsModels
using ForwardDiff
using Interpolations
using OSQP.MathOptInterfaceOSQP
using OSQP
import MathOptInterface
const MOI = MathOptInterface
using Parametron
using JLD2
import StaticArrays: SUnitRange
import DifferentialDynamicsModels: mod2piF, adiff
import Interpolations: GriddedInterpolation, Extrapolation
Parametron.Parameter(A::AbstractArray, model) = Parameter(identity, A, model)

include("math.jl")
include("vehicles.jl")
include("vehicle_dynamics.jl")
include("HJI_computation.jl")
include("HJI_Human.jl")
include("trajectories.jl")
include("model_predictive_control.jl")
include("decoupled_lat_long.jl")
include("coupled_lat_long.jl")
include("model_predictive_control.jl")
include("decoupled_lat_long.jl")
include("coupled_lat_long.jl")

const HJI_cache = HJICache(joinpath(@__DIR__, "../deps/BicycleCAvoid.jld2"));

In [2]:
trajectory = straight_trajectory(30., 5.) #(len, vel)

# Construct Decoupled MPC --> path
X1DMPC = DecoupledTrajectoryTrackingMPC(X1(), trajectory)
X1DMPC.current_state = BicycleState(0., 0., 0., 8., 0., 0.) #(X, Y, th, Ux, Uy, yawrate)
X1DMPC.current_control = BicycleControl(0., 3000., 0.) #(δ, Fxf, Fxr)

# Construct Coupled MPC --> trajectory
X1CMPC = CoupledTrajectoryTrackingMPC(X1(), trajectory, N_short=5, N_long=10)
X1CMPC.current_state = BicycleState(0., 0., 0., 8., 0., 0.)
X1CMPC.current_control = BicycleControl(0., 0., 0.)

# Determine Robot tracking type
tracking_mode = fill(:path)
tracking_mode[] = :traj  # Coupled MPC || :traj
robotMPC = (tracking_mode[] == :path ? X1DMPC : X1CMPC)

# Load HJI Cache
robotMPC.HJI_cache = HJI_cache;

In [8]:
# Construct Decoupled MPC --> HUMAN
# humanCMPC = CoupledTrajectoryTrackingMPC(X1(), trajectory)
robotMPC.other_car_state = SimpleCarState(-3., -2., 0.8, 4.) #(X, Y, th, V)

# Construct relative states
relative_state = HJIRelativeState(robotMPC.current_state, robotMPC.other_car_state) #(ΔX, ΔY, Δθ, Ux, Uy, V, r)

7-element HJIRelativeState{Float64}:
 -2.0
  3.0
  0.8
  8.0
  0.0
  4.0
  0.0

In [4]:
# Extract Value
V, ∇V = robotMPC.HJI_cache[relative_state]

(V = 0.7235913235125307, ∇V = [-0.532056, 0.758396, -1.03679, 0.0, 0.0, 0.0, 0.0])

In [9]:
_, _, t = path_coordinates(robotMPC.trajectory, robotMPC.current_state)
compute_time_steps!(robotMPC, t)
compute_linearization_nodes!(robotMPC)
update_QP!(robotMPC)
# solve!(robotMPC)

V = 0.7235913235125307ϵ = 10.0DIST = getfield(Main, Symbol("##19#21")){VehicleModel{Float64,BicycleModel{Float64}},HJIRelativeState{Float64},SArray{Tuple{7},Float64,1,7},SArray{Tuple{2},Float64,1,2}}(VehicleModel{Float64,BicycleModel{Float64}}(BicycleModel{Float64}(BicycleModelParams{Float64}(2.87, 1.4978360488798372, 1.372163951120163, 0.47, 9.80665, 1964.0, 2900.0, 0.92, 140000.0, 190000.0, 241.0, 25.1, 0.0)), LongitudinalActuationParams{Float64}(0.0, 1.0, 0.6, 0.4), ControlLimits{Float64}(5600.0, -16793.73299576057, 75000.0, 0.3141592653589793, 0.11321243771181404)), [-2.0, 3.0, 0.8, 8.0, 0.0, 4.0, 0.0], [-0.532056, 0.758396, -1.03679, 0.0, 0.0, 0.0, 0.0], [0.45285, -7.91528])

In [ ]:
s, e, _ = path_coordinates(robotMPC.trajectory, robotMPC.current_state)
u_next = get_next_control(robotMPC)

In [7]:
relative_state = HJIRelativeState(robotMPC.current_state, robotMPC.other_car_state)

7-element HJIRelativeState{Float64}:
 0.0
 0.0
 0.0
 8.0
 0.0
 0.0
 0.0

DIST = getfield(Main, Symbol("##19#21")){VehicleModel{Float64,BicycleModel{Float64}},HJIRelativeState{Float64},SArray{Tuple{7},Float64,1,7},SArray{Tuple{2},Float64,1,2}}(VehicleModel{Float64,BicycleModel{Float64}}(BicycleModel{Float64}(BicycleModelParams{Float64}(2.87, 1.4978360488798372, 1.372163951120163, 0.47, 9.80665, 1964.0, 2900.0, 0.92, 140000.0, 190000.0, 241.0, 25.1, 0.0)), LongitudinalActuationParams{Float64}(0.0, 1.0, 0.6, 0.4), ControlLimits{Float64}(5600.0, -16793.73299576057, 75000.0, 0.3141592653589793, 0.11321243771181404)), [-3.6, -5.0, 0.174533, 9.0, 0.0, 4.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0])

HUMAN = getfield(Main, Symbol("##89#90")){VehicleModel{Float64,BicycleModel{Float64}},HJIRelativeState_Human{Float64},SArray{Tuple{5},Float64,1,5},SArray{Tuple{1},Float64,1,1}}(VehicleModel{Float64,BicycleModel{Float64}}(BicycleModel{Float64}(BicycleModelParams{Float64}(2.87, 1.4978360488798372, 1.372163951120163, 0.47, 9.80665, 1964.0, 2900.0, 0.92, 140000.0, 190000.0, 241.0, 25.1, 0.0)), LongitudinalActuationParams{Float64}(0.0, 1.0, 0.6, 0.4), ControlLimits{Float64}(5600.0, -16793.73299576057, 75000.0, 0.3141592653589793, 0.11321243771181404)), [5.0, -3.6, 0.0, 0.174533, 9.0], [0.198853, -0.630596, 5.16971, -5.2784, -0.0155864], [1.0472])uR = [0.0, 0.0]